# index constituents merge

In [47]:
import pandas as pd
#3
# Paths to the CSV files
csv_file_path1 = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/index_constituents_data/constituents_stoxx_europe_600.csv'
csv_file_path2 = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/index_constituents_data/constituents_stoxx_europe_600_v2.csv'
csv_file_path3 = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/index_constituents_data/constituents_stoxx_europe_600_v3.csv'
csv_file_path4 = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/index_constituents_data/constituents_stoxx_europe_600_v4.csv'
output_file_path = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/index_constituents_data/constituents_stoxx_europe_600_merge.csv"

# Read CSV file
df1 = pd.read_csv(csv_file_path1)
df2 = pd.read_csv(csv_file_path2)
df3 = pd.read_csv(csv_file_path3)
df4 = pd.read_csv(csv_file_path4)

appended_df = pd.concat([df1, df2, df3, df4], ignore_index=True)
appended_df.columns = ["stock_RIC", "stock_name", "date"]
# Save the appended DataFrame to a new CSV file
appended_df.to_csv(output_file_path, index=False)

## Merge all the csv files to one data frame

In [65]:
import pandas as pd
import os
from datetime import datetime

xlsx_dir = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/index_proxi_data/raw_data'

merged_df = pd.DataFrame()

# Loop through all the files in the directory
for file_name in os.listdir(xlsx_dir):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(xlsx_dir, file_name)
        
        # Read the .xlsx file, assuming date is in cell B2 (second row, second column)
        date_cell = pd.read_excel(file_path, header=None, usecols="B", skiprows=1, nrows=1).iloc[0, 0]
        # Parse the date string to a datetime object and reformat it
        date_obj = datetime.strptime(date_cell, '%d-%b-%Y')
        formatted_date = date_obj.strftime('%Y-%m-%d')
        
        # Read the rest of the .xlsx file, skipping the first two rows (header and date)
        df = pd.read_excel(file_path, skiprows=2)
        df['Date'] = formatted_date
        
        merged_df = pd.concat([merged_df, df], ignore_index=True)

merged_df.columns = ["stock_RIC", "stock_name", "country", "weight", "shares", "change", "date"]
unique_df = merged_df.drop_duplicates(subset='stock_name')

output_file_path = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/index_proxi_data/RIC_mapping.xlsx'
unique_df.to_excel(output_file_path, index=False)

## handle missing data

In [46]:
import pandas as pd
import os
from datetime import datetime

merged_df['Date'] = pd.to_datetime(merged_df['Date'])  # Ensure 'Date' column is in datetime format

merged_df.sort_values('Date', inplace=True)

# Function to fill missing months
def fill_missing_months(data):
    unique_dates = data['Date'].unique()  # Extract unique dates
    min_date, max_date = min(unique_dates), max(unique_dates)
    all_dates = pd.date_range(start=min_date, end=max_date, freq='M').to_series()  # Generate all possible months
    
    last_valid_data = None
    results = []

    # Loop through each month in the generated date range
    for current_date in all_dates:
        if current_date in unique_dates:
            # If current month data is present, update last_valid_data
            last_valid_data = data[data['Date'] == current_date]
        elif last_valid_data is not None:
            # If no data for the current month, copy last valid data and change the date
            temp_data = last_valid_data.copy()
            temp_data['Date'] = current_date
            results.append(temp_data)
        else:
            # If no previous data is available (unlikely), continue without action
            continue

    # Concatenate all results with original data and re-sort
    if results:
        data = pd.concat([data] + results, ignore_index=True)
        data.sort_values('Date', inplace=True)

    return data

# Apply the function to fill missing months
filled_data = fill_missing_months(merged_df)
filled_data.columns = ["stock_RIC", "stock_name", "country", "weight", "shares", "change", "date"]

output_file_path = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/index_proxi_data/index_proxi_merge.csv'
filled_data.to_csv(output_file_path, index=False)

            Unnamed: 0                        Unnamed: 1      Unnamed: 2  \
35370         EDFp.NXT      ELECTRICITE DE FRANCE SA ORD          FRANCE   
35169            SDR.L                 SCHRODERS PLC ORD  UNITED KINGDOM   
35170  SCVbSEK.xbo^F14                     SCANIA AB ORD          SWEDEN   
35171           SCHP.S              SCHINDLER HOLDING AG     SWITZERLAND   
35172            SVT.L              SEVERN TRENT PLC ORD  UNITED KINGDOM   
...                ...                               ...             ...   
24793           BNZL.L                     BUNZL PLC ORD  UNITED KINGDOM   
24778            ABF.L  ASSOCIATED BRITISH FOODS PLC ORD  UNITED KINGDOM   
24779         ATCOa.ST                ATLAS COPCO AB ORD          SWEDEN   
24788          BICP.PA                SOCIETE BIC SA ORD          FRANCE   
24598           SMWH.L                  WH SMITH PLC ORD  UNITED KINGDOM   

      Unnamed: 3 Unnamed: 4 Unnamed: 5       Date  
35370   0.002209      50425       9

# index member dummy variable

In [6]:
import pandas as pd

filled_data = pd.read_csv('/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/index_proxi_data/index_proxi_merge.csv')
etf_data = filled_data[["stock_RIC", "stock_name", "date"]]
index_data = pd.read_csv("/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/index_constituents_data/constituents_stoxx_europe_600_merge.csv")

df = pd.concat([index_data, etf_data], ignore_index=True)

# Create pivot table
pivot_df = df.pivot_table(index='date', columns='stock_RIC', aggfunc='size', fill_value=0)

# Convert to long format
pivot_df.reset_index(inplace=True)
long_df = pivot_df.melt(id_vars=['date'], var_name='stock_RIC', value_name='member')

# Adjust the 'member' column to be binary
long_df['member'] = (long_df['member'] > 0).astype(int)

long_df.columns = ["date", "stock_RIC", "index_member"]

############
long_df['date'] = pd.to_datetime(long_df['date'])
long_df['date'] = long_df['date'].dt.date

########################################## changing dates from end of month to start of next month
def adjust_date_to_next_month_first(input_df, date_column):

    # Read the CSV file into a DataFrame
    df = input_df
    # Ensure the date column is in datetime format
    df[date_column] = pd.to_datetime(df[date_column])

    # Adjust the date to the first day of the following month
    df[date_column] = df[date_column] + pd.DateOffset(days=1)
    df[date_column] = df[date_column] - pd.to_timedelta(df[date_column].dt.day - 1, unit='d')

    # Write the updated DataFrame back to a new CSV file
    return df
    print(f"Dates in column '{date_column}' have been adjusted and saved to {output_file}")

long_df = adjust_date_to_next_month_first(long_df, "date")

##################################
def modify_and_append_date(df, original_date, new_date):
    selected_rows = df[df['date'] == pd.to_datetime(original_date)]

    # Modify the date of these rows
    selected_rows['date'] = pd.to_datetime(new_date)

    # Append these rows back to the original DataFrame
    return pd.concat([df, selected_rows])

long_df = modify_and_append_date(long_df, "2010-02-01", "2010-01-01")
long_df = long_df.sort_values(by=["stock_RIC", "date"], ascending=True)
# Save to CSV (optional)
file_path = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/index_constituents_data/formated_constituents_stoxx_europe_600.csv"
long_df.to_csv(file_path, index=False)

/var/folders/m9/n85vv59j4lj5g5nndc07m8t40000gn/T/ipykernel_99549/2999787493.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_rows['date'] = pd.to_datetime(new_date)
